## MD run analysis

See script [fragment_elaboration_scripts/md_run.py](https://github.com/matteoferla/Fragment-hit-follow-up-chemistry/blob/main/fragment_elaboration_scripts/md_run.py) for running the MD simulations.

In [ ]:
# ## Prep files

from openbabel import openbabel as ob

conv = ob.OBConversion()
conv.SetInFormat("sdf")
conv.SetOutFormat("mol")

obmol = ob.OBMol()
notatend = conv.ReadFile(obmol,"👾👾.sdf")
while notatend:
    obmol.CorrectForPH(7.4)
    obmol.AddHydrogens()
    conv.WriteFile(obmol, f'mols/{obmol.GetTitle()}.mol')
    # next
    obmol = ob.OBMol()
    notatend = conv.Read(obmol)

In [ ]:
# ## run the MD simulations

... # this is a Slurm job per hit

In [ ]:
# ## Combine and make RMSD summary
# Some jobs may have failed (see next cell). I rerun the missing ones.
# I doing RMSD against the crystal, which might not be perfect.
# I have the urge to switch to against the ensemble average, 
# but should not as it defeats the point: this is to assess fragment hits!

import pandas as pd
from pathlib import Path
from IPython.display import display
import plotly.express as px

info = {}
for path in Path('openmm_output').glob('*.pkl.gz'):
    #print(path.stem)
    rmsd_df = pd.read_pickle(path)
    info[path.stem.replace('.pkl', '')] = rmsd_df.RMSD.describe()

wobble = pd.DataFrame(info).transpose().sort_values('std')
_best = wobble.loc[(wobble['max'] < 2.) & (wobble['std'] < 0.5) & (wobble['50%'] < 1.)]
print(f'There are {len(wobble)} hits with RMSD data.'+\
      f'{len(_best)} are below 2.0 Å RMSD, with a median below 1.0 Å RMSD.')   
display(_best.sort_values('50%').head(10).round(2))
del _best
del info

px.histogram(wobble, '50%', title='RMSD Median Distribution')

In [ ]:
## Fix broken entries!
# TODO Add an echo in bash script with hit name, so I can see which ones failed.

import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis import rms
import plotly.express as px
import plotly.io as pio
pio.renderers.default='iframe'
import pandas as pd


template_filename = 'template2'
for mol_name in wobble.loc[wobble.RMSD_mean.isna()].index:
    prepped_filename = f'openmm_output/{template_filename}-{mol_name}.min.pdb'
    assert Path(prepped_filename).exists()
    traj_filename = f'openmm_output/{template_filename}-{mol_name}-{mol_name}.dcd'
    assert Path(traj_filename).exists()
    
    # Load the topology and trajectory
    u = mda.Universe(prepped_filename, traj_filename)
    
    # superpose frames
    aligner = align.AlignTraj(u, u, select="backbone", in_memory=True)
    aligner.run()
    
    # Set up the RMSD calculation
    selection = "resname LIG"
    R = rms.RMSD(u.select_atoms(selection), u.select_atoms(selection), select=selection, ref_frame=0)
    R.run()
    
    # Create a DataFrame for the RMSD values
    rmsd_df = pd.DataFrame({
        'Frame': R.results.rmsd[:, 1],
        'RMSD': R.results.rmsd[:, 2]
    })
    
    rmsd_df.to_pickle(f'openmm_output/{mol_name}.pkl.gz')

# rerun the summary cell

In [ ]:
# ## Visualise the best hits

from rdkit import Chem
from rdkit.Chem import Draw, AllChem
Draw.IPythonConsole.drawOptions.addAtomIndices = False

col = wobble.sort_values('50%').head(50)['50%']
legends = [f'{k.split("_")[0]} {v:.1}A' for k,v in col.items()]
mols = [Chem.MolFromMolFile(f'mols/{name}.mol') for name in col.index]
for mol in mols:
    AllChem.Compute2DCoords(mol)

Draw.MolsToGridImage(mols, legends=legends, molsPerRow=5)

## Residue and wobble

The RMSD is not a great metric.
It says what hits move _madly_ but at the small scale, it is not very informative,
because if the hit can sway with the residues nearby it is great,
but the RMSD will not capture that, but instead increase.

As a result the distance between the ligand and the residues could be more informative.

The following extracts the mean (ops) closest distance between the ligand and each protein residue, but does not go further.

In [ ]:
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Draw
Draw.IPythonConsole.drawOptions.addAtomIndices = True
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis import rms
import plotly.express as px
import plotly.io as pio
pio.renderers.default='iframe'
import pandas as pd

import numpy as np
from rdkit import Chem
from typing import Dict

# TODO use median instead of mean... and calculate the MAD as well
# or not? I would want to lose cases where distance outliers skew the data.
def calc_mean_closest_dist(universe: mda.Universe) -> Dict[str, float]:
    """
    Calculate the mean closest distance between the ligand and each protein residue.
    I need to switch the mean to np.median, and possibly add a better filter for the distances.
    
    :param universe: 
    :return: 
    """
    ligand = universe.select_atoms("resname LIG")
    nearby_protein_atoms = universe.select_atoms(f"protein")
    num_frames = len(universe.trajectory)
    num_ligand_atoms = len(ligand)
    num_protein_atoms = len(nearby_protein_atoms)
    # Get distances
    distances = np.zeros((num_frames, num_ligand_atoms, num_protein_atoms))
    for i, ts in enumerate(u.trajectory):
        dist_matrix = np.linalg.norm(
            ligand.positions[:, np.newaxis, :] - nearby_protein_atoms.positions[np.newaxis, :, :],
            axis=2
        )
        distances[i] = dist_matrix
    names = [f'{n}{i}:{c}' for n, i, c in zip(nearby_protein_atoms.resnames, nearby_protein_atoms.resnums, nearby_protein_atoms.chainIDs)]
    rearranged = {name: [] for name in names}
    for i, name in enumerate(names):
        rearranged[name].append(distances[:, :, i].min(axis=1))
    return {name: np.concatenate(lv).mean() for name, lv in rearranged.items()}

In [ ]:
distances: Dict[Dict[str]] = {}
for path in Path('openmm_output').glob('*.dcd'):
    temp_name = path.stem.split('-')[0]
    mol_name = path.stem.split('-')[1]
    u = mda.Universe(f'openmm_output/{temp_name}-{mol_name}.min.pdb', f'openmm_output/{temp_name}-{mol_name}-{mol_name}.dcd')
    aligner = align.AlignTraj(u, u, select="backbone", in_memory=True)
    aligner.run()
    distances[mol_name] = calc_mean_closest_dist(u)
    del u

dists = pd.DataFrame(distances).transpose()

# ## Temp hack for residue numbering
# neither atoms.resnums nor atoms.resids are correct
nearby_protein_atoms = u.select_atoms(f"protein")
wrong_names = [f'{n}{i}:{c}' for n, i, c in zip(nearby_protein_atoms.resnames, nearby_protein_atoms.resnums, nearby_protein_atoms.chainIDs)]
offsets = {'A': int(...), 'B': int(...)}
off_names = [f'{n}{i+offsets[c] - 1}:{c}' for n, i, c in zip(nearby_protein_atoms.resnames, nearby_protein_atoms.resnums, nearby_protein_atoms.chainIDs)]
dists=dists.rename(columns=dict(zip(wrong_names, off_names)))

# ## Combine with wobble
dists = dists[dists.columns[dists.min() < 4.0]]
new_names = {c: f'RMSD_{c}' for c in wobble.columns}
new_names['50%'] = 'RMSD_median'
df = pd.concat([dists, wobble.rename(columns=new_names)], axis=1)
del new_names

In [ ]:
# ## What residues have the least wobbly hits?

cutoff = 5
corr = df[[c for c, b in ((dists < cutoff).sum() >= 2).items() if b] + ['RMSD_mean', 'RMSD_std', 'RMSD_max',]].fillna(2).corr()
pd.concat([corr[['RMSD_mean', 'RMSD_std']].sort_values('RMSD_mean').round(2).RMSD_mean, (dists < cutoff).sum()], axis=1)

In [ ]:
# ## Plotting the distances
import plotly.express as px

m = df[[c for c in df.columns if ':' in c]].max().max()
blue_white = [
    [0.0, 'blue'],
    [3/m, 'gainsboro'],
    [1.0, 'white']
]
columns = sorted([c for c in df.columns if ':' in c], key=lambda c: int(c[3:-2])+{'A': 0,'B': 500}[c[-1]])

fig = px.imshow(df.sort_values('RMSD_mean')[columns].transpose(),
          color_continuous_scale=blue_white)
fig.update_layout(
    yaxis=dict(
        tickmode='auto',
        tickfont=dict(size=8),
        automargin=True,
    )
)
fig

In [ ]:
# ## tSNE Plotting
# This is very silly and meaningless

from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go

tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(dists)
tsne_df = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
tsne_df.index = dists.index
tsne_df['predictor'] = df['RMSD_mean'].fillna(2).values

centroids = {}
for resn in dists.columns:
    values = dists[resn].values
    mean_tSNE1 = tsne_df.loc[dists[resn] < 4.0].TSNE1.mean()
    mean_tSNE2 = tsne_df.loc[dists[resn] < 4.0].TSNE2.mean()
    centroids[resn] = [mean_tSNE1, mean_tSNE2]


fig = px.scatter(tsne_df, x='TSNE1', y='TSNE2', color='predictor', title='t-SNE Plot')
for resn, (cx, cy) in centroids.items():
    fig.add_trace(go.Scatter(
        x=[cx],
        y=[cy],
        mode='markers+text',
        marker=dict(symbol='x', size=12, color='black'),
        text=resn,
        textposition='top center',
        name=resn
    ))

fig

# Other snippets

In [ ]:
# get plot the RMSD of a single run
# mostly meaningless, bar for long state switching

from rdkit import Chem
from rdkit.Chem import Draw
Draw.IPythonConsole.drawOptions.addAtomIndices = True
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis import rms
import plotly.express as px
import plotly.io as pio
pio.renderers.default='iframe'
import pandas as pd

# Load the topology and trajectory
u = mda.Universe('openmm_output/👾👾.pdb', 'openmm_output/👾👾.dcd')

# superpose frames
aligner = align.AlignTraj(u, u, select="backbone", in_memory=True)
aligner.run()

# Set up the RMSD calculation
selection = "resname LIG"
R = rms.RMSD(u.select_atoms(selection), u.select_atoms(selection), select=selection, ref_frame=0)
R.run()

# Create a DataFrame for the RMSD values
rmsd_df = pd.DataFrame({
    'Frame': R.results.rmsd[:, 1],
    'RMSD': R.results.rmsd[:, 2]
})

# Plot the RMSD values using Plotly Express
fig = px.line(rmsd_df, x='Frame', y='RMSD', title='RMSD per Frame', labels={'Frame': 'Frame', 'RMSD': 'RMSD (Å)'})
fig.show()

In [ ]:
# ## Filtering by donor/acceptor atoms
# this needs work.

import numpy as np

distance_cutoff = 5.0

donor_atoms = [
    ('ALA', 'N'),
    ('ARG', 'N'), ('ARG', 'NE'), ('ARG', 'NH1'), ('ARG', 'NH2'),
    ('ASN', 'N'), ('ASN', 'ND2'),
    ('ASP', 'N'),
    ('CYS', 'N'),
    ('GLN', 'N'), ('GLN', 'NE2'),
    ('GLU', 'N'),
    ('GLY', 'N'),
    ('HIS', 'N'), ('HIS', 'ND1'), ('HIS', 'NE2'),
    ('ILE', 'N'),
    ('LEU', 'N'),
    ('LYS', 'N'), ('LYS', 'NZ'),
    ('MET', 'N'),
    ('PHE', 'N'),
    ('PRO', 'N'),
    ('SER', 'N'), ('SER', 'OG'),
    ('THR', 'N'), ('THR', 'OG1'),
    ('TRP', 'N'), ('TRP', 'NE1'),
    ('TYR', 'N'), ('TYR', 'OH'),
    ('VAL', 'N')
]

acceptor_atoms = [
    ('ALA', 'O'),
    ('ARG', 'O'), ('ARG', 'OXT'),
    ('ASN', 'O'), ('ASN', 'OXT'), ('ASN', 'OD1'),
    ('ASP', 'O'), ('ASP', 'OXT'), ('ASP', 'OD1'), ('ASP', 'OD2'),
    ('CYS', 'O'), ('CYS', 'OXT'),
    ('GLN', 'O'), ('GLN', 'OXT'), ('GLN', 'OE1'),
    ('GLU', 'O'), ('GLU', 'OXT'), ('GLU', 'OE1'), ('GLU', 'OE2'),
    ('GLY', 'O'), ('GLY', 'OXT'),
    ('HIS', 'O'), ('HIS', 'OXT'), ('HIS', 'ND1'), ('HIS', 'NE2'),
    ('ILE', 'O'), ('ILE', 'OXT'),
    ('LEU', 'O'), ('LEU', 'OXT'),
    ('LYS', 'O'), ('LYS', 'OXT'),
    ('MET', 'O'), ('MET', 'OXT'),
    ('PHE', 'O'), ('PHE', 'OXT'),
    ('PRO', 'O'), ('PRO', 'OXT'),
    ('SER', 'O'), ('SER', 'OXT'), ('SER', 'OG'),
    ('THR', 'O'), ('THR', 'OXT'), ('THR', 'OG1'),
    ('TRP', 'O'), ('TRP', 'OXT'), ('TRP', 'NE1'),
    ('TYR', 'O'), ('TYR', 'OXT'), ('TYR', 'OH'),
    ('VAL', 'O'), ('VAL', 'OXT')
]

# smarts is a valid selection language but this is waaaay faster
# https://www.daylight.com/dayhtml_tutorials/languages/smarts/smarts_examples.html#H_BOND
ligand = u.select_atoms("resname LIG")
mol = ligand.convert_to("RDKIT")
acceptor = Chem.MolFromSmarts('[!$([#1,#6,F,Cl,Br,I,o,s,nX3,#7v5,#15v5,#16v4,#16v6,*+1,*+2,*+3])]')
donor = Chem.MolFromSmarts('[!H0;#7,#8,#9]')
# checking:
# gly = Chem.MolFromSmiles('[NH3+]CC(=O)[O-]')
#print( gly.GetSubstructMatches(acceptor), gly.GetSubstructMatches(donor) )
get_name = lambda i: mol.GetAtomWithIdx(i).GetPDBResidueInfo().GetName().strip()
namesselector = lambda names: '('+(' or '.join(f'name {n}' for n in names))+')'
nsel = namesselector([get_name(i[0]) for i in mol.GetSubstructMatches(acceptor)])
ligand = u.select_atoms(f"resname LIG and {nsel}")

# Select nearby protein donor atoms
nsel = namesselector(set([n for r, n in donor_atoms]))
nearby_protein_atoms = u.select_atoms(f"protein and {nsel} and around {distance_cutoff} group ligand", ligand=ligand)

# Get the numbers
num_frames = len(u.trajectory)
num_ligand_atoms = len(ligand)
num_protein_atoms = len(nearby_protein_atoms)
# Get distances
distances = np.zeros((num_frames, num_ligand_atoms, num_protein_atoms))
for i, ts in enumerate(u.trajectory):
    dist_matrix = np.linalg.norm(
        ligand.positions[:, np.newaxis, :] - nearby_protein_atoms.positions[np.newaxis, :, :],
        axis=2
    )
    distances[i] = dist_matrix

Inspection in PyMol after loading the two parts

```pymol
intra_fit byres resn LIG around 4, 1
show sticks, not resn HOH
hide lines, resn HOH
zoom resn LIG
color coral, resn LIG and element C
color turquoise, polymer and element C and chain A
color teal, polymer and element C and chain B
color cyan, polymer and element C and chain C
```
